In [1]:
# Importing the necessary packages
import pandas as pd 
import sklearn
from sklearn.random_projection import GaussianRandomProjection
import numpy as np
from matplotlib import pyplot as plt


In [2]:
import math
from statistics import mean

convert to matrix from dataframe

In [3]:
data_raw = pd.read_csv('datasets/after_preproccessing.csv')
data_raw.drop(columns=data_raw.columns[0], axis=1,  inplace=True)
label_data = data_raw['is_patient']
label_data=label_data.replace(2,0)
data_raw.drop(columns=['is_patient'], axis=1 , inplace=True)
X_Matrix = data_raw.to_numpy()
data_raw.to_csv('datasets/train.csv')
labels = label_data.to_numpy()

In [4]:
X_Matrix.shape

(583, 11)

Random Projections ( Gaussian RP )
Reducing dimensions from 11 components to n_components = 8 
Here hyperparameters : n_components (8 for now )
K1=8

In [5]:
# Random data and its transformation

X_random = X_Matrix
gauss_data = GaussianRandomProjection(random_state=0 , n_components=8)
X_transformed = gauss_data.fit_transform(X_random)

# Get the size of the transformed data
print('Shape of transformed data is: ' + str(X_transformed.shape))



Shape of transformed data is: (583, 8)


Getting matrix R (random matrix )

In [6]:
Random_Projection_matrix=gauss_data.components_

In [7]:
X_transformed.shape

(583, 8)

Recommended parameters from paper:
delta : 0.0001
epsilon : [8,6,4,2,1]
Privacy budget :
b=80% for Random Projection
1-b% for SVD ( covariace )
k1=8
k2=0.6 *d


Noise matrix M1
M1 ~ N(0,variance1) 
Hyperparameter : standard deviation1 sd1 (variance1)
std_p = 1 / sqrt(k1)
delta1 
epsilon1
Z for now 5

In [8]:
delta1=8e-5
epsilon1=3.2
k1=8
std_p = 1/math.sqrt(k1)
Z=5e-5

calculating variance 1 for M1 matrix (noise matrix for random projection)

In [9]:
std1=Z * std_p * math.sqrt(k1 + 2*math.sqrt(k1*math.log(2/delta1)) + 2*math.log(2/delta1) ) * math.sqrt(2* ( math.log(1/(2*delta1))  +epsilon1 ) ) /(epsilon1)

In [10]:
print(pow(std1,2))

3.370952377012233e-08


generating M1 matrix of 583 x 8

In [11]:
M1= np.random.normal(0,pow(std1,2) , size=(583,8))

Adding noise to random projection matrix
Finaly:

Random projection matrix with noise .

In [12]:
P_new = X_transformed + M1
print(P_new.shape)

(583, 8)


In [13]:
print(P_new[10])
print(X_transformed[10])
print(M1[10])

[217.19524132  99.47115252 -89.38733977  63.61546393  12.1264454
 -61.16374565 -12.08894932 -73.59204914]
[217.19524133  99.47115254 -89.38733977  63.61546393  12.12644543
 -61.16374558 -12.08894935 -73.59204915]
[-9.13637582e-09 -1.38876759e-08 -2.25095669e-09 -4.94084920e-09
 -2.69786067e-08 -7.27436595e-08  3.38056782e-08  8.21208281e-09]


covariance matrix 

var2 = Z2**2    x  sqrt( 2*ln1.25/delta2 ) / epsilon2

N (0, Z
2p
2 ln 1.25/δ2/2)

In [14]:
Xc=np.cov(X_Matrix.T)
print(Xc.shape)

(11, 11)


Adding noise to covariance matrix

In [15]:
Z2=1e-3
delta2 = 2e-5
epsilon2 = 0.8
var2 = pow(Z2,2) * math.sqrt( 2* math.log( 1.25 ) / delta2 ) / epsilon2

t1 = np.random.normal(0 , var2 , size=(11,11) )
M2 = (t1 + t1.T)/2

XC_new = Xc + M2

print(XC_new.shape)

(11, 11)


SVD of covariance matrix

In [16]:
u , s , vt = np.linalg.svd(XC_new,full_matrices=True)
print(u.shape , s.shape , vt.shape)

(11, 11) (11,) (11, 11)


selection top K columns from V`

k2 = 0.6*d

In [17]:
k2 = 6
n=u.shape[0]
u_new=[]
for i in range(n):
    u_new.append(u[i][:k2])
u_k2 =np.array(u_new)
print(u_k2.shape)

(11, 6)


Getting final data set .

used : Moore-Penrose pseudoinverse
x_final = p' (u_k2.R)+ u_k2.T

In [18]:
X_final = np.matmul (  np.matmul( P_new , np.linalg.pinv( np.matmul(u_k2.T, Random_Projection_matrix.T   ) )  )   ,u_k2.T )

Tesing average on age of patients in data set. 

In [19]:
original_age_list =[]
transformed_age_list =[]
for i in range(583):
    original_age_list.append(X_Matrix[i][0])
    transformed_age_list.append(X_final[i][0])

print(f"Average original age is :",mean(original_age_list) )
print(f"Average Transformed age is :",mean(transformed_age_list) )



Average original age is : 44.74614065180103
Average Transformed age is : 46.76000666110246


Prediction "Patient" using Random Forest Classifier . 
It is giving the best classification accuracy among all models 

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_final, labels, test_size=0.2, random_state=42)

def fit_and_score_model(X, y, model_clf, X_v, y_v):
    model_clf.fit(X, y)
    return model_clf.score(X_v, y_v)

In [21]:
from sklearn.ensemble import RandomForestClassifier

rfc_clf = RandomForestClassifier(n_estimators=150, n_jobs=-1, random_state=42, max_depth=200,)
r=fit_and_score_model(X_train, y_train, rfc_clf, X_val, y_val)
print("accuracy is ",r)

accuracy is  0.7863247863247863
